In [ ]:
import sys
sys.path.insert(0, '..//src')
sys.path.insert(0, '..//src//data')
sys.path.insert(0, '..//src//features')
sys.path.insert(0, '..//src//models')

import global_func as gf
import data_preprocessing
import predict
import fit_model

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 

config_dir = '..\\config\\'
data_dir = '..\\' + gf.read_config(config_dir=config_dir, section='DIR', key='DATA_PROCESSED')
models_dir = '..\\' + gf.read_config(config_dir=config_dir, section='DIR', key='MODELS')
remodel_dir = '..\\' + gf.read_config(config_dir=config_dir, section='DIR', key='DATA_REMODEL')


DataPrep = data_preprocessing.DataPreprocessing()
model = gf.load_from_pkl(filename=f'{models_dir}sarimax.pkl')
scaler = gf.load_from_pkl(filename=f'{models_dir}scaler.pkl')
Predictor = predict.Predictor(model=model, scaler=scaler)
gf.save_as_pkl(obj=Predictor, filename=f'{models_dir}predictor.pkl', compress=6)

# Load Clean Data

In [ ]:
df_raw = gf.load_from_pkl(f'{data_dir}df_raw.pkl')
df_train = gf.load_from_pkl(filename=f'{data_dir}df_train_05.pkl')
df_valid = gf.load_from_pkl(filename=f'{data_dir}df_valid_05.pkl')
df_test = gf.load_from_pkl(filename=f'{data_dir}df_test_05.pkl')

df_train.name = 'df_train'
df_valid.name = 'df_valid'
df_test.name = 'df_test'

#gf.plotting_line_all(dataframe=df_train)

# Modelling

## Fitting Model

In [ ]:
model = fit_model.modelling(train_dataframe=df_train)

## Model Summary

In [ ]:
print(model.summary())

## Save to PKL

In [ ]:
#gf.create_pkl(obj=model, pkl=r'..\models\pkl\sarimax(0,0,0)(2,1,0)(12).pkl')

# End to End Process

## Step 1

In [ ]:
prior = [100.23,200.23,300.23,400.23,500.23,440.23,330.23,202.23,110.239]
current = [10100.669,202.669,303.669,604.669,50500.669,449.669,339.669,292.669,119.669]

df = Predictor.parsing_data(prior=prior, current=current)
print(Predictor.input_dict)
df

## Step 2

In [ ]:
# Maybe Unecessary
df_2 = DataPrep.resampling(dataframe=df, interval='B', resampling_method='median', fillna_method='ffill')
df_2

## Step 3

In [ ]:
df_3 = DataPrep.enriching(dataframe=df_2)
df_3

## Step 4

In [ ]:
df_seasonal = gf.load_from_pkl(f'{remodel_dir}value_for_seasonal.pkl')
df_4 = DataPrep.enriching_seasonal(dataframe=df_3, df_seasonal=df_seasonal)
df_4

## Step 5

In [ ]:
pkl = f'{remodel_dir}list_non_stationary_cols.pkl'
df_5 = DataPrep.stationary_transform(dataframe=df_4, non_stationary_cols_pkl=pkl)
df_5

## Step 6

In [ ]:
df_value_for_outlier = gf.load_from_pkl(f'{remodel_dir}value_for_outlier.pkl')
df_6 = DataPrep.outlier_treatment_batch(dataframe=df_5, df_value_for_outlier=df_value_for_outlier)
df_6

## Step 7

In [ ]:
df_7 = DataPrep.std_scaler_transform(dataframe=df_6, scaler=scaler)
df_7

## Step 8

In [ ]:
df_8 = Predictor.forecasting(dataframe=df_7, name='df_8')
df_8

# Predicting

In [ ]:
df_forecast_valid = Predictor.forecasting(dataframe=df_valid, name='pred_valid')
df_forecast_test = Predictor.forecasting(dataframe=df_test, name='pred_test')

In [ ]:
df_forecast_test[['lq45_inv','pred_lq45_inv']].plot(figsize =(15,4))
df_forecast_valid[['lq45_inv','pred_lq45_inv']].plot(figsize =(15,4))
plt.show()

## Predict Real

In [ ]:
for i in range(1,11):
    i_to = -1 * i
    i_from = i_to - 1
    df_real = gf.load_from_pkl(filename=f'{data_dir}df_test_05.pkl')[i_from:i_to]
    df_real_pred = Predictor.forecasting(dataframe=df_real, name='pred_real')


    print(df_real_pred)